In [2]:
import mercury as mr
app = mr.App(title="Mercury Pro new user 🚀", 
             description="Add access to Mercury Pro private repository to a new user")

mercury.App

In [3]:
_ = mr.Note("Please provide email address used during purchase and GitHub username.")

Please provide email address used during purchase and GitHub username.

In [4]:
subscriber_email = mr.Text(label="Email used during purchase", value="")
new_github_user = mr.Text(label="GitHub username", value="")

mercury.Text

mercury.Text

# Mercury Pro new user

Thanks for choosing Mercury Pro! It is a commercial license for <a href="https://github.com/mljar/mercury" target="_blank">Mercury</a> framework. It has additional features, dedicated support and allows white labeling.

Steps get access to the Mercury Pro private repository ([github.com/mljar/mercury-pro](https://github.com/mljar/mercury-pro)):

1. Please purchase [Mercury Pro license](https://mljar.com/pricing).
2. Please provide email address used during purchase and your GitHub username in the left sidebar and click `Run`.
3. The notebook will check if your email is in the subscribers list.
4. If you are active subscriber to Mercury Pro license your GitHub username will be added to collaborators list for Mercury Pro repository.

In the case of any questions please contact us by email `contact@mljar.com`.

In [ ]:
import os
import json
import requests
from dotenv import load_dotenv

In [ ]:
_ = load_dotenv()

In [ ]:
ok = True  # flag to control execution

In [ ]:
if subscriber_email == "" or "@" not in subscriber_email:
    print("Please provide email address")
    ok = False
if new_github_user == "":
    print("Please provide GitHub username")
    ok = False

In [ ]:
if ok:
    response = requests.get(f"https://api.github.com/users/{new_github_user}").json()
    if response.get("message", "") == "Not Found":
        print(f"The username {new_github_user} does not exist in the GitHub.")
        ok = False

In [ ]:
PADDLE_VENDOR_ID = os.environ.get("PADDLE_VENDOR_ID")
PADDLE_AUTH_CODE = os.environ.get("PADDLE_AUTH_CODE")
GITHUB_TOKEN = os.environ.get("GITHUB_TOKEN")

In [ ]:
if PADDLE_VENDOR_ID is None:
    print("Please provide PADDLE_VENDOR_ID environment variable")
    ok = False
if PADDLE_AUTH_CODE is None:
    print("Please provide PADDLE_AUTH_CODE environment variable")
    ok = False
if GITHUB_TOKEN is None:
    print("Please provide GITHUB_TOKEN environment variable")
    ok = False

In [ ]:
if ok:
    data = {"vendor_id": PADDLE_VENDOR_ID, "vendor_auth_code": PADDLE_AUTH_CODE}
    response = requests.post(
        "https://vendors.paddle.com/api/2.0/subscription/users", data=data
    )
    if response.status_code != 200:
        print("Cant load list of avialable users. Sorry!")
        ok = False

In [ ]:
active_emails = []
if ok:
    if response.status_code == 200:
        subscribers = response.json()["response"]
        for sub in subscribers:
            if sub["state"] == "active":
                active_emails += [sub["user_email"]]

In [ ]:
if ok:
    if not subscriber_email in active_emails:
        print(f"The address {subscriber_email} is not active subscriber")
        ok = False

In [ ]:
if ok:
    response = requests.put(
        f"https://api.github.com/repos/mljar/mercury-pro/collaborators/{new_github_user}",
        headers={
            "Authorization": f"token {GITHUB_TOKEN}",
            "Accept": "application/vnd.github+json",
        },
        data=json.dumps({"permission": "pull"}),
    )

    if response.status_code == 204:
        print(
            "Success, you should be able to access https://github.com/mljar/mercury-pro"
        )
    else:
        print(f"Problem with adding user {new_user} to mercury-pro repository.")
        print("Please contact us by email: contact@mljar.com")